In [91]:
#!/usr/bin/env python

import numpy as np
import scipy
import json
from sklearn import model_selection as cross_validation
import re
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import shuffle
def parse_file(filename):

    def parse(filename):
        tweet = []
        print(filename)
        with open(filename) as f:
            for line in f:
#                 print(line)
                tweet.append(line)
        return tweet

    tweets = parse(filename)
    gt = []
    plots = []
    idx = []
    for i,twt in enumerate(tweets):
        tweet = twt.split(':')
#         print(tweet)
        genre = tweet[0]
        tweet_txt = tweet[1]
#         tweet_txt = re.sub(r"@\w+","", tweet[1])
#         tweet_txt = ' '.join(tweet_txt.split(' ')[3:])
        
        if 'NUM' in genre:
            plots.append(tweet_txt)
            gt.append(0)
            idx.append(i)
        elif 'LOC' in genre:
            plots.append(tweet_txt)
            gt.append(1)
            idx.append(i)
        elif 'HUM' in genre:
            plots.append(tweet_txt)
            gt.append(2)
            idx.append(i)
        elif 'DESC' in genre:
            plots.append(tweet_txt)
            gt.append(3)
            idx.append(i)
        elif 'ENTY' in genre:
            plots.append(tweet_txt)
            gt.append(4)
            idx.append(i)
        elif 'ABBR' in genre:
            plots.append(tweet_txt)
            gt.append(5)
            idx.append(i)
        else:
            continue  

    print('len of data',len(plots))
    return np.array(plots), np.array(gt)

def split_data(plots, y):
    np.random.seed(1234)
#     num_sample = np.shape(X)[0]
    num_test = 500
    plots, y  = shuffle( plots, y, random_state = 25)
#     X_test = X[0:num_test,:]
#     X_train = X[num_test:, :]
    plots_train = plots[num_test:]
    plots_test = plots[0:num_test]

    y_test = y[0:num_test]
    y_train = y[num_test:]

    # split dev/test
    test_ratio = 0.05
    y_tr, y_te, plots_tr, plots_te =cross_validation.train_test_split(y_train, plots_train, test_size = test_ratio, random_state=25)

    return np.array(y_te), np.array(y_test), plots_te, plots_test

In [92]:
data_path='data/trec/'
plots, labels = parse_file(data_path+'all.txt')
y_valid, y_test, plots_valid, plots_test =  split_data(plots,labels)

data/trec/all.txt
len of data 5965


In [86]:
plots[14]

'what sprawling u.s. state boasts the most airports ?\n'

In [81]:
labels[0]

3

In [ ]:
x = [('did','you'), ('develop', 'kyun')]

for i in x:
    plots[0] = plots[0].replace(i[0],i[1])
print(plots[0])

In [41]:
mapp

{0: 'NUM', 1: 'LOC', 2: 'HUM', 3: 'DESC', 4: 'ENTY', 5: 'ABBR'}

In [82]:
plots[0]

mapp = {0:'NUM',1:'LOC',2:'HUM',3:'DESC', 4:'ENTY', 5:'ABBR'}

mapp[1]

'LOC'

In [90]:
import spacy

nlp = spacy.load("en_core_web_sm")
# doc = nlp(plots)
phrases = set()
writestr = ''
for i in range(len(labels)):
    doc = nlp(str(plots[i]))
    rep_arr = []
    plots[i] = plots[i].replace("'",'').replace("`",'').replace(":","")
    plots[i] = plots[i].strip()
#     print('inside',plots[i])
    for e in doc.ents:
        rep_arr.append((e.text, e.label_))
#         print(e.text, e.start_char, e.end_char, e.label_)
#     print(rep_arr)
    if len(rep_arr) > 0:
        for rep in rep_arr:
            replac = rep[0] + " " + rep[1]
            plots[i] = plots[i].replace(rep[0], replac )
    writestr += str(mapp[labels[i]]) + ":" + str(plots[i]) +'\n'
with open('data/trec/trec_ent_replaced_or.txt','w') as f:
    f.write(writestr)

In [74]:
x = "when did spielberg direct `` jaws ''" 
print(x.replace("'",""))

when did spielberg direct `` jaws 


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
def create_ngram_features(words, n=2):
    ngram_vocab = ngrams(words, n)
    my_dict = dict([(ng, True) for ng in ngram_vocab])
    return my_dict

stoplist = set(stopwords.words("english"))

def create_word_features(words):
    useful_words = [word for word in words if word not in stoplist] 
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

In [ ]:
cl = []
for x,y in zip(plots_valid, y_valid):
    words = x.split()
    cl.append(((create_ngram_features(words)),y))
test = []
for x,y in zip(plots_test, y_test):
    words = x.split()
    test.append((create_ngram_features(words),y))

In [ ]:
cl[0]

In [ ]:
# from textblob.classifiers import NaiveBayesClassifier
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
clf = NaiveBayesClassifier.train(cl)

In [ ]:
nltk.classify.util.accuracy(clf, test)

In [ ]:
def show_most_informative_features_in_list(classifier, n=10):
    """
    Return a nested list of the "most informative" features 
    used by the classifier along with it's predominant labels
    """
    cpdist = classifier._feature_probdist       # probability distribution for feature values given labels
    feature_list = []
    for (fname, fval) in classifier.most_informative_features(n):
        def labelprob(l):
            return cpdist[l, fname].prob(fval)
        labels = sorted([l for l in classifier._labels if fval in cpdist[l, fname].samples()], 
                        key=labelprob)
        feature_list.append([fname, labels[-1]])
    return feature_list

In [ ]:
feats = show_most_informative_features_in_list(clf,50)
for i in feats:
    print(i[0])

In [ ]:
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
for i in plots:
    blob = TextBlob(i, np_extractor=extractor)
    print(blob.noun_phrases)

In [ ]:
c = 0
with open('np_trec.txt','w') as f:
    for i in phrases:
        f.write(i)
    

In [ ]:
ph = []
for i in phrases:
    ph.append(str(i))

In [ ]:
vocab = []
with open('np_trec.txt','r') as f:
    for i in f.readlines():
        vocab.append(i)

In [ ]:
len(vocab)


In [ ]:
vocab1 = ["what"]#,"does the","what 's","mean ?","are the","stand for","what does","for ?","in ?","other What","was the","what was","do you","does a","in the","the most","the first","is the","on a","did the","name ?","of the","name the","can i","is a","in what","by the","chancellor of","of ?","were the","from the","into the","What are","australia ?","book to","call the","do italians","italians call","the tallest","to help","was hitler","has the"]

In [5]:
def mytokenizer(text):
    return text.split()

In [9]:
vectorizer = CountVectorizer(min_df=1, decode_error='ignore', strip_accents='ascii', ngram_range=(1,2))
X = vectorizer.fit_transform(plots)

In [10]:
np.sum(X,0).shape

(1, 32310)

In [11]:
valid_feats = np.where(np.sum(X,0)> 0)[1]

In [12]:
valid_feats

array([    0,     1,     2, ..., 32307, 32308, 32309])

In [13]:
X = X[:,valid_feats]

In [ ]:
def mytokenizer(text):
    return text.split()

In [14]:
def prune_features(val_primitive_matrix, train_primitive_matrix, thresh=0.001):
    val_sum = np.sum(np.abs(val_primitive_matrix),axis=0)
    train_sum = np.sum(np.abs(train_primitive_matrix),axis=0)

    #Only select the indices that fire more than 1% for both datasets
    train_idx = np.where((train_sum >= thresh*np.shape(train_primitive_matrix)[0]))[0]
    val_idx = np.where((val_sum >= thresh*np.shape(val_primitive_matrix)[0]))[0]
    common_idx = list(set(train_idx) & set(val_idx))
    return common_idx

In [ ]:
# vocab = {'abbreviation':0,'actor':1,'actress':2,'address':3,'age':4,'alias':5,'amount':6,'are':7,'around':8,'at':9,'ate':10,'book':11,'build':12,'built':13,'by':14,'called':15,'can':16,'capital':17,'captain':18,'citizen':19,'close':20,'company':21,'composed':22,'could':23,'country':24,'date':25,'day':26,'demands':27,'describe':28,'did':29,'discovered':30,'division':31,'do':32,'doctor':33,'does':34,'does ':35,'doesn':36,'engineer':37,'enumerate':38,'explain':39,'far':40,'fastener':41,'fastener ':42,'fear':43,'for':44,'found':45,'from':46,'game':47,'gamer':48,'governs':49,'group':50,'groups':51,'guarded':52,'hero':53,'hours':54,'how':55,'human':56,'hypertension':57,'in':58,'instance':59,'invented':60,'is':61,'is ':62,'island':63,'kind':64,'king':65,'latitude':66,'latitude ':67,'lawyer':68,'leader':69,'leads':70,'list':71,'lived':72,'lives':73,'located':74,'long':75,'longitude':76,'made':77,'man':78,'many':79,'mean':80,'meant':81,'minute':82,'model':83,'month':84,'movie':85,'much':86,'name':87,'name ':88,'nationalist':89,'near':90,'nicknamed':91,'novel':92,'number':93,'object':94,'of':95,'old':96,'organization':97,'origin':98,'out':99,'owner':100,'owns':101,'part':102,'patent':103,'pays':104,'percentage':105,'person':106,'play':107,'played':108,'player':109,'poet':110,'population':111,'portrayed':112,'president':113,'queen':114,'ratio':115,'run':116,'seconds':117,'served':118,'shall':119,'share':120,'short':121,'should':122,'should ':123,'situated':124,'slept':125,'small':126,'speed':127,'stand':128,'star':129,'studied':130,'study ':131,'surname':132,'surrounds':133,'take':134,'tall':135,'team':136,'teams':137,'tetrinet':138,'the':139,'thing':140,'through':141,'time':142,'to':143,'trust':144,'unusual':145,'used':146,'using':147,'various':148,'was':149,'was ':150,'watched':151,'what':152,'what ':153,'when':154,'where':155,'where ':156,'which':157,'who':158,'who ':159,'why':160,'wide':161,'will':162,'woman':163,'worked':164,'would':165,'year':166,'you':167}
# vocab = {'name a':0,'how does':1,'how to':2,'how can':3,'how should':4,'how would':5,'how could':6,'how will':7,'how do':8,'what is':9,'what fastener':10,'how do you':11,'who person':12,'who man':13,'who woman':14,'who human':15,'who president':16,'what person':17,'what man':18,'what woman':19,'what human':20,'what president':21,'how much':22,'how many':23,'what kind':24,'what amount':25,'what number':26,'what percentage':27,'capital of':28,'why does':29,'why should':30,'why shall':31,'why could':32,'why would':33,'why will':34,'why can':35,'why do':36,'composed from':37,'composed through':38,'composed using':39,'composed by':40,'composed of':41,'made from':42,'made through':43,'made using':44,'made by':45,'made of':46,'where island':47,'which island':48,'what island':49,'who owner':50,'who leads':51,'who governs':52,'who pays':53,'who owns':54,'what is tetrinet':55,'who found':56,'who discovered':57,'who made':58,'who built':59,'who build':60,'who invented':61,'why doesn':62,'used for':63,'when did':64,'when do':65,'when does':66,'when was':67,'how old':68,'how far':69,'how long':70,'how tall':71,'how wide':72,'how short':73,'how small':74,'how close':75,'fear of':76,'explain can':77,'describe can':78,'who worked':79,'who lived':80,'who guarded':81,'who watched':82,'who played':83,'who ate':84,'who slept':85,'who portrayed':86,'who served':87,'what part':88,'what division':89,'what ratio':90,'who is':91,'who will':92,'who was':93,'what do':94,'what does':95,'enumerate the various':96,'list out the various':97,'name the various':98,'at which age':99,'at which year':100,'at how many age':101,'at how many year':102,'at what age':103,'at what year':104,'in which age':105,'in which year':106,'in how many age':107,'in how many year':108,'in what age':109,'in what year':110,'which play':111,'which game':112,'which movie':113,'which book':114,'what play':115,'what game':116,'what movie':117,'what book':118,'which is':119,'which will':120,'which are':121,'which was':122,'who are':123,'by how much':124,'by how many':125,'where was':126,'where is':127,'studied':128,'patent':129,'who':130,'man':131,'swoman':132,'human':133,'person':134,'where':135,'stand':136,'what':137,'mean':138,'meant':139,'called':140,'unusual':141,'origin':142,'country':143,'queen':144,'king':145,'year':146,'novel':147,'speed':148,'abbreviation':149,'percentage':150,'share':151,'number':152,'population':153,'explain':154,'describe':155,'located':156,'thing':157,'instance':158,'object':159,'demands':160,'take':161,'leader':162,'citizen':163,'captain':164,'nationalist':165,'hero':166,'actor':167,'actress':168,'star':169,'gamer':170,'player':171,'lawyer':172,'president':173,'lives':174,'latitude':175,'longitude':176,'alias':177,'nicknamed':178}
vectorizer = CountVectorizer(vocabulary=vocab, tokenizer = mytokenizer , ngram_range=(1,2))#, stop_words='english')
X = vectorizer.fit_transform(plots)

In [ ]:
np.sum(X[:,32])

In [ ]:
X.shape

In [ ]:
np.sum(X[:,149])

In [ ]:
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
        train_ground, val_ground, test_ground,\
        train_plots, val_plots, test_plots = split_data(X, plots, labels)

In [ ]:
val_sum = np.sum(np.abs(val_primitive_matrix),axis=0)
train_sum = np.sum(np.abs(train_primitive_matrix),axis=0)

In [ ]:
# valid_feats = np.where(np.sum(X,0)> 2)[1]
# X = X[:,valid_feats]


train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
train_ground, val_ground, test_ground,\
train_plots, val_plots, test_plots = split_data(X, plots, labels)

#Prune Feature Space
common_idx = prune_features(val_primitive_matrix, train_primitive_matrix)

# return train_primitive_matrix[:,common_test_primitive_matrixidx], val_primitive_matrix[:,common_idx], test_primitive_matrix[:,common_idx], \
#         np.array(train_ground), np.array(val_ground), np.array(test_ground), vectorizer, valid_feats, common_idx, \
#     train_plots, val_plots, test_plots

In [ ]:
common_idx

In [ ]:
[vectorizer.get_feature_names()[i] for i in common_idx ]

In [ ]:
len([vectorizer.get_feature_names()[valid_feats[i]] for i in common_idx ])

In [ ]:
class DataLoader(object):
    """ A class to load in appropriate numpy arrays
    """

    def prune_features(self, val_primitive_matrix, train_primitive_matrix, thresh=0.01):
        val_sum = np.sum(np.abs(val_primitive_matrix),axis=0)
        train_sum = np.sum(np.abs(train_primitive_matrix),axis=0)

        #Only select the indices that fire more than 1% for both datasets
        train_idx = np.where((train_sum >= thresh*np.shape(train_primitive_matrix)[0]))[0]
        val_idx = np.where((val_sum >= thresh*np.shape(val_primitive_matrix)[0]))[0]
        common_idx = list(set(train_idx) & set(val_idx))

        return common_idx
    def load_data(self, dataset, data_path='/home/ayusham/Semi_Supervised_LFs/Data/TREC/'):
     
        plots, labels = parse_file(data_path+'all.txt')
        
        #Featurize Plots  
        vectorizer = CountVectorizer(min_df=1, binary=True, stop_words='english', \
            decode_error='ignore', strip_accents='ascii', ngram_range=(1,2))
        X = vectorizer.fit_transform(plots)
        valid_feats = np.where(np.sum(X,0)> 2)[1]
        X = X[:,valid_feats]

#         Split Dataset into Train, Val, Test
        train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
        train_ground, val_ground, test_ground,\
        train_plots, val_plots, test_plots = split_data(X, plots, labels)

        #Prune Feature Space
        common_idx = self.prune_features(val_primitive_matrix, train_primitive_matrix)
#         print('common_idx',len(common_idx))
#         return train_primitive_matrix[:,common_idx], val_primitive_matrix[:,common_idx], test_primitive_matrix[:,common_idx],             np.array(train_ground), np.array(val_ground), np.array(test_ground),train_plots, val_plots, test_plots

        return train_primitive_matrix[:,common_idx], val_primitive_matrix[:,common_idx], test_primitive_matrix[:,common_idx], \
                np.array(train_ground), np.array(val_ground), np.array(test_ground), vectorizer, valid_feats, common_idx, \
            train_plots, val_plots, test_plots



In [101]:
# arguments - dataset(1) mode(random/all/normal)(2) model(dt/lr/nn)(3) cardinality(4) num_of_loops(5)
# save directory (6)


# python generic_generate_labels.py imdb normal dt 1 26 imdb_val2.5_sup5_dt1

import numpy as np
import matplotlib.pyplot as plt
import sys
from program_synthesis.heuristic_generator import HeuristicGenerator
from program_synthesis.synthesizer import Synthesizer
import pickle
import os
import warnings
from sklearn import model_selection as cross_validation
warnings.filterwarnings("ignore")

dataset= sys.argv[1]
print('dataset is ', dataset)
loader_file = "data.trec_loader"

import importlib

load = importlib.import_module(loader_file)

dl = load.DataLoader()
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, train_ground,\
    val_ground, test_ground, _, _, _, _,_,_ = dl.load_data(dataset=dataset, split_val = 0.1)


dataset is  -f
/home/ayusham/auto_lfs/reef/data/trec/all.txt
len of data 5965
4774 [    0     5     6 ... 36844 36855 36856]
common_idx 1094


In [102]:
train_ground[:int(len(val_ground)/2)]

array([-1,  3,  1, -1,  2, -1,  2,  3,  3,  3,  4,  3,  3,  3,  3,  2,  2,
        3,  4,  1, -1,  3, -1,  3,  4, -1,  4,  4,  1,  2,  1,  1,  3,  1,
        4,  4, -1,  4,  4, -1,  2,  4,  2,  3,  2,  4,  4,  4,  1,  3,  1,
        2,  3,  3,  2, -1,  4, -1, -1,  2,  2, -1, -1,  2,  5,  2, -1,  3,
        2,  2,  4, -1,  1,  5, -1,  2, -1,  3,  1,  4,  1,  4,  2,  2,  3,
        3,  1,  3,  3,  3,  1,  2,  3,  2,  3,  4,  4, -1,  1,  1, -1,  2,
        1,  1,  2, -1,  2, -1,  3,  1, -1,  1,  4,  3, -1,  1,  2,  4,  2,
        4,  3,  1,  4,  2,  3,  1,  3, -1,  4, -1, -1,  2,  3,  3,  3, -1,
        3,  3,  2,  2,  3,  2,  4,  4,  2, -1,  4,  1, -1, -1, -1, -1,  2,
        2,  4,  1, -1,  1,  1, -1,  3,  4,  3, -1,  4,  4, -1,  1,  1,  1,
        4,  3,  3, -1,  2,  3, -1,  4, -1,  2,  2, -1,  1,  3,  3,  1, -1,
        1,  2,  1,  3,  2,  1,  2,  3,  4,  2, -1,  1,  4, -1,  4,  3, -1,
        2,  4, -1, -1,  3,  3,  3,  2,  1,  3, -1,  2,  2,  4,  4,  3,  4,
       -1,  4,  4, -1, -1

In [100]:
train_ground[:int(len(val_ground)/2)]

array([-1,  3,  1, -1,  2, -1,  2,  3,  3,  3,  4,  3,  3,  3,  3,  2,  2,
        3,  4,  1, -1,  3, -1,  3,  4, -1,  4,  4,  1,  2,  1,  1,  3,  1,
        4,  4, -1,  4,  4, -1,  2,  4,  2,  3,  2,  4,  4,  4,  1,  3,  1,
        2,  3,  3,  2, -1,  4, -1, -1,  2,  2, -1, -1,  2,  5,  2, -1,  3,
        2,  2,  4, -1,  1,  5, -1,  2, -1,  3,  1,  4,  1,  4,  2,  2,  3,
        3,  1,  3,  3,  3,  1,  2,  3,  2,  3,  4,  4, -1,  1,  1, -1,  2,
        1,  1,  2, -1,  2, -1,  3,  1, -1,  1,  4,  3, -1,  1,  2,  4,  2,
        4,  3,  1,  4,  2,  3,  1,  3, -1,  4, -1, -1,  2,  3,  3,  3, -1,
        3,  3,  2,  2,  3,  2,  4,  4,  2, -1,  4,  1, -1, -1, -1, -1,  2,
        2,  4,  1, -1,  1,  1, -1,  3,  4,  3, -1,  4,  4, -1,  1,  1,  1,
        4,  3,  3, -1,  2,  3, -1,  4, -1,  2,  2, -1,  1,  3,  3,  1, -1,
        1,  2,  1,  3,  2,  1,  2,  3,  4,  2, -1,  1,  4, -1,  4,  3, -1,
        2,  4, -1, -1,  3,  3,  3,  2,  1,  3, -1,  2,  2,  4,  4,  3,  4,
       -1,  4,  4, -1, -1

In [95]:
val_ground

array([ 2,  1, -1,  4,  2, -1,  3,  3,  3,  4,  3,  1,  1,  4,  1,  4,  1,
        1,  2,  3,  3, -1,  2,  2,  1, -1,  4,  2, -1,  4,  4,  2,  3,  2,
       -1,  2,  2,  2,  2,  2,  3,  1,  3,  2, -1,  1,  3,  4,  4,  2,  4,
       -1,  4, -1,  1,  4,  4,  4,  3,  1,  1, -1,  4, -1, -1,  4,  4,  2,
        4,  4,  4,  4,  4,  5,  3,  3,  1,  5,  2,  2, -1,  2,  4,  2,  4,
        4,  4, -1,  1,  1,  2,  4, -1,  3, -1,  1, -1, -1,  2,  4, -1, -1,
       -1,  4,  4,  4,  1,  3,  3, -1,  2, -1, -1,  3,  1,  2,  3, -1, -1,
        1,  1,  4,  3,  3,  2,  1,  1,  4,  1,  4,  2,  2,  3, -1,  2,  2,
       -1,  4,  2, -1,  4,  2, -1, -1,  3,  3, -1,  1,  4,  3,  4,  3,  5,
        1,  2,  3,  4,  2,  2,  2,  3,  3,  4,  4,  4,  4,  1, -1,  2,  2,
        2,  2,  3, -1,  1,  3,  1,  3,  2,  3,  2,  2,  4,  2,  4,  1,  3,
        3,  4,  2,  3,  2,  2,  5,  3,  2, -1,  2,  4,  2, -1,  1,  3,  1,
        2,  1,  1,  4,  2,  4,  1,  2,  4,  5,  4,  4,  4,  4,  2,  4,  4,
        3,  3,  2,  3,  1